<a href="https://colab.research.google.com/github/soham7707/finmodel-lab/blob/main/Bearish_Test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LATTICE STRUCTURE FOR BEARISH PERIOD:

Lstm Beta model combined, where the weights of each has been decided by linear regression model


In [ ]:
pip install yfinance arch statsmodels matplotlib pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 8.7 MB/s eta 0:00:00


In [ ]:
def recommend_stocks_lstm_beta(
    stocks, index_symbol, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense

    results = []

    # Index returns
    index_data = yf.download(index_symbol, start=start_date, end=end_date)['Close']
    index_returns = np.log(index_data / index_data.shift(1)).dropna()

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 20:
                print(f"⚠️ Not enough data for {stock}")
                continue

            combined = pd.concat([stock_returns, index_returns], axis=1).dropna()
            combined.columns = ['Stock', 'Market']

            # --- Beta estimation ---
            X = sm.add_constant(combined['Market'])
            model = sm.OLS(combined['Stock'], X).fit()
            beta = model.params['Market']

            # --- LSTM volatility estimation ---
            scaler = MinMaxScaler()
            scaled_returns = scaler.fit_transform(stock_returns.values.reshape(-1, 1))

            # Prepare LSTM data
            X_lstm, y_lstm = [], []
            window = 10
            for i in range(window, len(scaled_returns)):
                X_lstm.append(scaled_returns[i - window:i])
                y_lstm.append(scaled_returns[i])
            X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

            # LSTM model
            lstm_model = Sequential()
            lstm_model.add(LSTM(units=50, return_sequences=False, input_shape=(X_lstm.shape[1], 1)))
            lstm_model.add(Dense(1))
            lstm_model.compile(optimizer='adam', loss='mean_squared_error')
            lstm_model.fit(X_lstm, y_lstm, epochs=10, batch_size=8, verbose=0)

            # Predict and compute volatility
            predictions = lstm_model.predict(X_lstm, verbose=0)
            predicted_returns = scaler.inverse_transform(predictions)
            predicted_vol = np.std(predicted_returns)

            avg_return = np.mean(stock_returns)
            std_return = np.std(stock_returns)
            #sharpe_ratio = (avg_return / std_return) if std_return != 0 else 0

            results.append({
                'Stock': stock,
                'Beta': round(beta, 4),
                'LSTM_Volatility': round(predicted_vol, 4),
                'Avg_Return': round(avg_return, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)

    # Drop rows with missing values
    df.dropna(subset=['Beta', 'LSTM_Volatility', 'Avg_Return'], inplace=True)


    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()

    # --- Learn weights via regression ---
    reg = LinearRegression()
    X = df[['Beta', 'LSTM_Volatility']].values
    y = df['Avg_Return'].values

    reg.fit(X, y)
    learned_weights = reg.coef_

    beta_weight, lstm_weight = learned_weights
    print(f"\n📈 Learned Weights from Regression: Beta = {beta_weight:.4f}, LSTM_Vol = {lstm_weight:.4f}")

    # --- Compute risk score using learned weights ---
    df['Risk_Score'] = beta_weight * df['Beta'] + lstm_weight * df['LSTM_Volatility']
    df = df.sort_values(by='Risk_Score')

    print(df[['Stock', 'Beta', 'LSTM_Volatility', 'Avg_Return', 'Risk_Score']].to_string(index=False))


    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f}, Beta: {row['Beta']}, Vol: {row['LSTM_Volatility']}")

    return top


recommend_stocks_lstm_beta(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    index_symbol="^NSEI",
    start_date="2022-09-01",  # Bearish period start
    end_date="2022-12-31",    # Bearish period end
    top_n=5
)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3800: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When usin


📈 Learned Weights from Regression: Beta = 0.0001, LSTM_Vol = -0.2503
        Stock   Beta  LSTM_Volatility  Avg_Return  Risk_Score
    ZOMATO.NS 2.2029           0.0043     -0.0006   -0.000858
      INFY.NS 1.2140           0.0039      0.0005   -0.000856
    MARUTI.NS 0.8566           0.0031     -0.0009   -0.000691
     WIPRO.NS 0.9991           0.0030     -0.0005   -0.000652
ULTRACEMCO.NS 1.0092           0.0030      0.0006   -0.000651
 SUNPHARMA.NS 0.4088           0.0027      0.0017   -0.000635
ASIANPAINT.NS 0.3966           0.0025     -0.0013   -0.000586
  RELIANCE.NS 1.1060           0.0026     -0.0001   -0.000541
       TCS.NS 0.8844           0.0024      0.0005   -0.000513
       ITC.NS 0.8304           0.0021      0.0005   -0.000443
BAJAJ-AUTO.NS 0.8656           0.0021     -0.0014   -0.000440
 ICICIBANK.NS 0.9622           0.0021      0.0002   -0.000430
     TITAN.NS 0.7984           0.0019     -0.0001   -0.000396
        LT.NS 1.0514           0.0020      0.0010   -0.000396


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3800: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


,Stock,Beta,LSTM_Volatility,Avg_Return,Risk_Score
14,ZOMATO.NS,2.2029,0.0043,-0.0006,-0.000858
0,INFY.NS,1.2140,0.0039,0.0005,-0.000856
6,MARUTI.NS,0.8566,0.0031,-0.0009,-0.000691
13,WIPRO.NS,0.9991,0.0030,-0.0005,-0.000652
10,ULTRACEMCO.NS,1.0092,0.0030,0.0006,-0.000651


Lstm Beta model combined, where the weights of each has been decided by linear regression model (target variable is sharpe ratio)

In [ ]:
def recommend_stocks_lstm_beta(
    stocks, index_symbol, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense

    results = []

    # Index returns
    index_data = yf.download(index_symbol, start=start_date, end=end_date)['Close']
    index_returns = np.log(index_data / index_data.shift(1)).dropna()

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 20:
                print(f"⚠️ Not enough data for {stock}")
                continue

            combined = pd.concat([stock_returns, index_returns], axis=1).dropna()
            combined.columns = ['Stock', 'Market']

            # --- Beta estimation ---
            X = sm.add_constant(combined['Market'])
            model = sm.OLS(combined['Stock'], X).fit()
            beta = model.params['Market']

            # --- LSTM volatility estimation ---
            scaler = MinMaxScaler()
            scaled_returns = scaler.fit_transform(stock_returns.values.reshape(-1, 1))

            # Prepare LSTM data
            X_lstm, y_lstm = [], []
            window = 10
            for i in range(window, len(scaled_returns)):
                X_lstm.append(scaled_returns[i - window:i])
                y_lstm.append(scaled_returns[i])
            X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

            # LSTM model
            lstm_model = Sequential()
            lstm_model.add(LSTM(units=50, return_sequences=False, input_shape=(X_lstm.shape[1], 1)))
            lstm_model.add(Dense(1))
            lstm_model.compile(optimizer='adam', loss='mean_squared_error')
            lstm_model.fit(X_lstm, y_lstm, epochs=10, batch_size=8, verbose=0)

            # Predict and compute volatility
            predictions = lstm_model.predict(X_lstm, verbose=0)
            predicted_returns = scaler.inverse_transform(predictions)
            predicted_vol = np.std(predicted_returns)

            avg_return = np.mean(stock_returns)
            sharpe_ratio = avg_return / predicted_vol if predicted_vol != 0 else 0

            results.append({
                'Stock': stock,
                'Beta': round(beta, 4),
                'LSTM_Volatility': round(predicted_vol, 4),
                'Avg_Return': round(avg_return, 4),
                'Sharpe_Ratio': round(sharpe_ratio, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)

    # Drop rows with missing values
    df.dropna(subset=['Beta', 'LSTM_Volatility', 'Sharpe_Ratio'], inplace=True)

    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()

    # --- Learn weights via regression on Sharpe Ratio ---
    reg = LinearRegression()
    X = df[['Beta', 'LSTM_Volatility']].values
    y = df['Sharpe_Ratio'].values
    reg.fit(X, y)
    learned_weights = reg.coef_

    beta_weight, lstm_weight = learned_weights
    print(f"\n📈 Learned Weights from Regression: Beta = {beta_weight:.4f}, LSTM_Vol = {lstm_weight:.4f}")

    # --- Compute risk score using learned weights ---
    df['Risk_Score'] = beta_weight * df['Beta'] + lstm_weight * df['LSTM_Volatility']
    df = df.sort_values(by='Risk_Score')

    print(df[['Stock', 'Beta', 'LSTM_Volatility', 'Avg_Return', 'Sharpe_Ratio', 'Risk_Score']].to_string(index=False))

    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f}, Sharpe Ratio: {row['Sharpe_Ratio']:.4f}, Beta: {row['Beta']}, Vol: {row['LSTM_Volatility']}")

    return top


In [ ]:
recommend_stocks_lstm_beta(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    index_symbol="^NSEI",
    start_date="2022-09-01",  # Bearish period start
    end_date="2022-12-31",    # Bearish period end
    top_n=5
)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When


📈 Learned Weights from Regression: Beta = 0.2726, LSTM_Vol = -210.7452
        Stock   Beta  LSTM_Volatility  Avg_Return  Sharpe_Ratio  Risk_Score
ULTRACEMCO.NS 1.0092           0.0031      0.0006        0.1842   -0.378251
    MARUTI.NS 0.8566           0.0029     -0.0009       -0.3034   -0.377693
 SUNPHARMA.NS 0.4088           0.0023      0.0017        0.7254   -0.373295
    ZOMATO.NS 2.2029           0.0046     -0.0006       -0.1367   -0.369024
ASIANPAINT.NS 0.3966           0.0022     -0.0013       -0.5922   -0.355545
      INFY.NS 1.2140           0.0031      0.0005        0.1562   -0.322432
     WIPRO.NS 0.9991           0.0027     -0.0005       -0.1936   -0.296706
BAJAJ-AUTO.NS 0.8656           0.0023     -0.0014       -0.6248   -0.248793
       ITC.NS 0.8304           0.0022      0.0005        0.2372   -0.237312
  RELIANCE.NS 1.1060           0.0024     -0.0001       -0.0256   -0.204346
       TCS.NS 0.8844           0.0021      0.0005        0.2430   -0.201520
 ICICIBANK.NS 0.

,Stock,Beta,LSTM_Volatility,Avg_Return,Sharpe_Ratio,Risk_Score
10,ULTRACEMCO.NS,1.0092,0.0031,0.0006,0.1842,-0.378251
6,MARUTI.NS,0.8566,0.0029,-0.0009,-0.3034,-0.377693
7,SUNPHARMA.NS,0.4088,0.0023,0.0017,0.7254,-0.373295
14,ZOMATO.NS,2.2029,0.0046,-0.0006,-0.1367,-0.369024
5,ASIANPAINT.NS,0.3966,0.0022,-0.0013,-0.5922,-0.355545


BETA GARCH MODEL COMBINED:

In [ ]:
def recommend_stocks_beta_garch(
    stocks, index_symbol, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from arch import arch_model
    from sklearn.linear_model import LinearRegression

    results = []

    # Index returns
    index_data = yf.download(index_symbol, start=start_date, end=end_date)['Close']
    index_returns = np.log(index_data / index_data.shift(1)).dropna()

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 30:
                print(f"⚠️ Not enough data for {stock}")
                continue

            combined = pd.concat([stock_returns, index_returns], axis=1).dropna()
            combined.columns = ['Stock', 'Market']

            # --- Beta estimation ---
            X = sm.add_constant(combined['Market'])
            model = sm.OLS(combined['Stock'], X).fit()
            beta = model.params['Market']

            # --- GARCH volatility estimation ---
            garch_model = arch_model(stock_returns * 100, vol='GARCH', p=1, q=1)
            garch_result = garch_model.fit(disp='off')
            garch_volatility = np.mean(np.sqrt(garch_result.conditional_volatility)) / 100  # scale back

            avg_return = np.mean(stock_returns)
            sharpe_ratio = avg_return / garch_volatility if garch_volatility != 0 else 0

            results.append({
                'Stock': stock,
                'Beta': round(beta, 4),
                'GARCH_Volatility': round(garch_volatility, 4),
                'Avg_Return': round(avg_return, 4),
                'Sharpe_Ratio': round(sharpe_ratio, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)

    # Drop rows with missing values
    df.dropna(subset=['Beta', 'GARCH_Volatility', 'Sharpe_Ratio'], inplace=True)

    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()

    # --- Learn weights via regression on Sharpe Ratio ---
    reg = LinearRegression()
    X = df[['Beta', 'GARCH_Volatility']].values
    y = df['Sharpe_Ratio'].values
    reg.fit(X, y)
    learned_weights = reg.coef_

    beta_weight, garch_weight = learned_weights
    print(f"\n📉 Learned Weights from Regression: Beta = {beta_weight:.4f}, GARCH_Vol = {garch_weight:.4f}")

    # --- Compute risk score using learned weights ---
    df['Risk_Score'] = beta_weight * df['Beta'] + garch_weight * df['GARCH_Volatility']
    df = df.sort_values(by='Risk_Score')

    print(df[['Stock', 'Beta', 'GARCH_Volatility', 'Avg_Return', 'Sharpe_Ratio', 'Risk_Score']].to_string(index=False))

    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f}, Sharpe Ratio: {row['Sharpe_Ratio']:.4f}, Beta: {row['Beta']}, Vol: {row['GARCH_Volatility']}")

    return top

In [ ]:
recommend_stocks_beta_garch(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    index_symbol="^NSEI",
    start_date="2022-09-01",  # Bearish period start
    end_date="2022-12-31",
    top_n=5
)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********


📉 Learned Weights from Regression: Beta = 0.0862, GARCH_Vol = -26.1391
        Stock   Beta  GARCH_Volatility  Avg_Return  Sharpe_Ratio  Risk_Score
    ZOMATO.NS 2.2029            0.0175     -0.0006       -0.0356   -0.267638
ASIANPAINT.NS 0.3966            0.0111     -0.0013       -0.1175   -0.255974
 SUNPHARMA.NS 0.4088            0.0105      0.0017        0.1584   -0.239239
     TITAN.NS 0.7984            0.0117     -0.0001       -0.0096   -0.237039
    MARUTI.NS 0.8566            0.0112     -0.0009       -0.0775   -0.218955
      INFY.NS 1.2140            0.0122      0.0005        0.0398   -0.214302
       ITC.NS 0.8304            0.0109      0.0005        0.0475   -0.213371
ULTRACEMCO.NS 1.0092            0.0114      0.0006        0.0506   -0.211036
     WIPRO.NS 0.9991            0.0112     -0.0005       -0.0473   -0.206678
       TCS.NS 0.8844            0.0108      0.0005        0.0465   -0.206105
BAJAJ-AUTO.NS 0.8656            0.0106     -0.0014       -0.1358   -0.202497
 ICI

,Stock,Beta,GARCH_Volatility,Avg_Return,Sharpe_Ratio,Risk_Score
14,ZOMATO.NS,2.2029,0.0175,-0.0006,-0.0356,-0.267638
5,ASIANPAINT.NS,0.3966,0.0111,-0.0013,-0.1175,-0.255974
7,SUNPHARMA.NS,0.4088,0.0105,0.0017,0.1584,-0.239239
12,TITAN.NS,0.7984,0.0117,-0.0001,-0.0096,-0.237039
6,MARUTI.NS,0.8566,0.0112,-0.0009,-0.0775,-0.218955


BETA LSTM GARCH MODEL COMBINED

In [ ]:
def recommend_stocks_beta_garch_lstm(
    stocks, index_symbol, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from arch import arch_model
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense

    results = []

    # Index returns
    index_data = yf.download(index_symbol, start=start_date, end=end_date)['Close']
    index_returns = np.log(index_data / index_data.shift(1)).dropna()

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 30:
                print(f"⚠️ Not enough data for {stock}")
                continue

            combined = pd.concat([stock_returns, index_returns], axis=1).dropna()
            combined.columns = ['Stock', 'Market']

            # --- Beta estimation ---
            X = sm.add_constant(combined['Market'])
            model = sm.OLS(combined['Stock'], X).fit()
            beta = model.params['Market']

            # --- GARCH volatility estimation ---
            garch_model = arch_model(stock_returns * 100, vol='GARCH', p=1, q=1)
            garch_result = garch_model.fit(disp='off')
            garch_vol = np.mean(np.sqrt(garch_result.conditional_volatility)) / 100  # rescale

            # --- LSTM volatility estimation ---
            scaler = MinMaxScaler()
            scaled_returns = scaler.fit_transform(stock_returns.values.reshape(-1, 1))

            X_lstm, y_lstm = [], []
            window = 10
            for i in range(window, len(scaled_returns)):
                X_lstm.append(scaled_returns[i - window:i])
                y_lstm.append(scaled_returns[i])
            X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

            lstm_model = Sequential()
            lstm_model.add(LSTM(units=50, return_sequences=False, input_shape=(X_lstm.shape[1], 1)))
            lstm_model.add(Dense(1))
            lstm_model.compile(optimizer='adam', loss='mean_squared_error')
            lstm_model.fit(X_lstm, y_lstm, epochs=10, batch_size=8, verbose=0)

            predictions = lstm_model.predict(X_lstm, verbose=0)
            predicted_returns = scaler.inverse_transform(predictions)
            lstm_vol = np.std(predicted_returns)

            # --- Sharpe Ratio ---
            avg_return = np.mean(stock_returns)
            avg_vol = np.mean([garch_vol, lstm_vol])
            sharpe_ratio = avg_return / avg_vol if avg_vol != 0 else 0

            results.append({
                'Stock': stock,
                'Beta': round(beta, 4),
                'GARCH_Volatility': round(garch_vol, 4),
                'LSTM_Volatility': round(lstm_vol, 4),
                'Avg_Return': round(avg_return, 4),
                'Sharpe_Ratio': round(sharpe_ratio, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)
    df.dropna(subset=['Beta', 'GARCH_Volatility', 'LSTM_Volatility', 'Sharpe_Ratio'], inplace=True)

    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()

    # --- Learn weights via regression on Sharpe Ratio ---
    reg = LinearRegression()
    X = df[['Beta', 'GARCH_Volatility', 'LSTM_Volatility']].values
    y = df['Sharpe_Ratio'].values
    reg.fit(X, y)
    beta_w, garch_w, lstm_w = reg.coef_

    print(f"\n📊 Learned Weights → Beta = {beta_w:.4f}, GARCH_Vol = {garch_w:.4f}, LSTM_Vol = {lstm_w:.4f}")

    # --- Risk Score ---
    df['Risk_Score'] = (
        beta_w * df['Beta'] +
        garch_w * df['GARCH_Volatility'] +
        lstm_w * df['LSTM_Volatility']
    )

    df = df.sort_values(by='Risk_Score')

    print(df[['Stock', 'Beta', 'GARCH_Volatility', 'LSTM_Volatility', 'Avg_Return', 'Sharpe_Ratio', 'Risk_Score']].to_string(index=False))

    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f}, Sharpe: {row['Sharpe_Ratio']:.4f}, Beta: {row['Beta']}, GARCH Vol: {row['GARCH_Volatility']}, LSTM Vol: {row['LSTM_Volatility']}")

    return top

In [ ]:
recommend_stocks_beta_garch_lstm(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    index_symbol="^NSEI",
    start_date="2022-09-01",  # Bearish period start
    end_date="2022-12-31",
    top_n=5
)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When


📊 Learned Weights → Beta = 0.1301, GARCH_Vol = -49.5008, LSTM_Vol = 29.5634
        Stock   Beta  GARCH_Volatility  LSTM_Volatility  Avg_Return  Sharpe_Ratio  Risk_Score
    ZOMATO.NS 2.2029            0.0175           0.0044     -0.0006       -0.0570   -0.449509
ASIANPAINT.NS 0.3966            0.0111           0.0023     -0.0013       -0.1946   -0.429852
     TITAN.NS 0.7984            0.0117           0.0017     -0.0001       -0.0168   -0.425001
 SUNPHARMA.NS 0.4088            0.0105           0.0025      0.0017        0.2555   -0.392651
       ITC.NS 0.8304            0.0109           0.0024      0.0005        0.0777   -0.360542
    MARUTI.NS 0.8566            0.0112           0.0029     -0.0009       -0.1229   -0.357201
BAJAJ-AUTO.NS 0.8656            0.0106           0.0022     -0.0014       -0.2255   -0.347024
     WIPRO.NS 0.9991            0.0112           0.0027     -0.0005       -0.0760   -0.344569
       TCS.NS 0.8844            0.0108           0.0027      0.0005        0.

,Stock,Beta,GARCH_Volatility,LSTM_Volatility,Avg_Return,Sharpe_Ratio,Risk_Score
14,ZOMATO.NS,2.2029,0.0175,0.0044,-0.0006,-0.0570,-0.449509
5,ASIANPAINT.NS,0.3966,0.0111,0.0023,-0.0013,-0.1946,-0.429852
12,TITAN.NS,0.7984,0.0117,0.0017,-0.0001,-0.0168,-0.425001
7,SUNPHARMA.NS,0.4088,0.0105,0.0025,0.0017,0.2555,-0.392651
8,ITC.NS,0.8304,0.0109,0.0024,0.0005,0.0777,-0.360542


LSTM GARCH MODEL COMBINED

In [ ]:
def recommend_stocks_lstm_garch(
    stocks, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    from arch import arch_model
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense

    results = []

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 30:
                print(f"⚠️ Not enough data for {stock}")
                continue

            # --- GARCH volatility estimation ---
            garch_model = arch_model(stock_returns * 100, vol='GARCH', p=1, q=1)
            garch_result = garch_model.fit(disp='off')
            garch_vol = np.mean(np.sqrt(garch_result.conditional_volatility)) / 100

            # --- LSTM volatility estimation ---
            scaler = MinMaxScaler()
            scaled_returns = scaler.fit_transform(stock_returns.values.reshape(-1, 1))

            X_lstm, y_lstm = [], []
            window = 10
            for i in range(window, len(scaled_returns)):
                X_lstm.append(scaled_returns[i - window:i])
                y_lstm.append(scaled_returns[i])
            X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

            lstm_model = Sequential()
            lstm_model.add(LSTM(units=50, return_sequences=False, input_shape=(X_lstm.shape[1], 1)))
            lstm_model.add(Dense(1))
            lstm_model.compile(optimizer='adam', loss='mean_squared_error')
            lstm_model.fit(X_lstm, y_lstm, epochs=10, batch_size=8, verbose=0)

            predictions = lstm_model.predict(X_lstm, verbose=0)
            predicted_returns = scaler.inverse_transform(predictions)
            lstm_vol = np.std(predicted_returns)

            # --- Sharpe Ratio ---
            avg_return = np.mean(stock_returns)
            avg_vol = np.mean([garch_vol, lstm_vol])
            sharpe_ratio = avg_return / avg_vol if avg_vol != 0 else 0

            results.append({
                'Stock': stock,
                'GARCH_Volatility': round(garch_vol, 4),
                'LSTM_Volatility': round(lstm_vol, 4),
                'Avg_Return': round(avg_return, 4),
                'Sharpe_Ratio': round(sharpe_ratio, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)
    df.dropna(subset=['GARCH_Volatility', 'LSTM_Volatility', 'Sharpe_Ratio'], inplace=True)

    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()

    # --- Learn weights via regression on Sharpe Ratio ---
    reg = LinearRegression()
    X = df[['GARCH_Volatility', 'LSTM_Volatility']].values
    y = df['Sharpe_Ratio'].values
    reg.fit(X, y)
    garch_w, lstm_w = reg.coef_

    print(f"\n📊 Learned Weights → GARCH_Vol = {garch_w:.4f}, LSTM_Vol = {lstm_w:.4f}")

    # --- Risk Score ---
    df['Risk_Score'] = (
        garch_w * df['GARCH_Volatility'] +
        lstm_w * df['LSTM_Volatility']
    )

    df = df.sort_values(by='Risk_Score')

    print(df[['Stock', 'GARCH_Volatility', 'LSTM_Volatility', 'Avg_Return', 'Sharpe_Ratio', 'Risk_Score']].to_string(index=False))

    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f}, Sharpe: {row['Sharpe_Ratio']:.4f}, GARCH Vol: {row['GARCH_Volatility']}, LSTM Vol: {row['LSTM_Volatility']}")

    return top

In [ ]:
recommend_stocks_lstm_garch(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    start_date="2022-09-01",  # Bearish period start
    end_date="2022-12-31",    # Bearish period end
    top_n=5
)


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as th


📊 Learned Weights → GARCH_Vol = -5.1115, LSTM_Vol = -26.8900
        Stock  GARCH_Volatility  LSTM_Volatility  Avg_Return  Sharpe_Ratio  Risk_Score
    ZOMATO.NS            0.0175           0.0045     -0.0006       -0.0567   -0.210457
      INFY.NS            0.0122           0.0034      0.0005        0.0622   -0.153787
ULTRACEMCO.NS            0.0114           0.0032      0.0006        0.0792   -0.144319
    MARUTI.NS            0.0112           0.0031     -0.0009       -0.1217   -0.140608
     WIPRO.NS            0.0112           0.0027     -0.0005       -0.0761   -0.129852
  RELIANCE.NS            0.0111           0.0026     -0.0001       -0.0090   -0.126652
ASIANPAINT.NS            0.0111           0.0024     -0.0013       -0.1935   -0.121274
 SUNPHARMA.NS            0.0105           0.0024      0.0017        0.2585   -0.118207
BAJAJ-AUTO.NS            0.0106           0.0023     -0.0014       -0.2235   -0.116029
        LT.NS            0.0109           0.0022      0.0010        

,Stock,GARCH_Volatility,LSTM_Volatility,Avg_Return,Sharpe_Ratio,Risk_Score
14,ZOMATO.NS,0.0175,0.0045,-0.0006,-0.0567,-0.210457
0,INFY.NS,0.0122,0.0034,0.0005,0.0622,-0.153787
10,ULTRACEMCO.NS,0.0114,0.0032,0.0006,0.0792,-0.144319
6,MARUTI.NS,0.0112,0.0031,-0.0009,-0.1217,-0.140608
13,WIPRO.NS,0.0112,0.0027,-0.0005,-0.0761,-0.129852
